
# Install dependencies: if not working just manually install in terminal using 

!pip install {insert library name}

In order for the code to work, you need to make sure you include the correct point to the local server below:

In [31]:
import os
import openai
import pandas as pd
import xlsxwriter


# *** TODO *** 
openai.api_base = "http://localhost:1688/v1" 
openai.api_key = "" # no need for an API key

def chat_with_llm(prompt):
    response = openai.ChatCompletion.create(
        model = "gpt-4-0613",
        messages = [{"role": "user", "content":prompt}]
    )

    return response.choices[0].message.content.strip()

data = pd.read_csv('data/data_v4.csv') #load in data

subset = data.iloc[0:50] #random sample of 100 complaints


In [32]:

role_textgen = """You are a customer of a large commercial bank such as Wells Fargo, J.P. Morgan Chase, Bank of America, or Citibank. You have had bad experiences and will 
now be voicing your frustration online by writing complaints to your commercial bank. Be creative. Don't be afraid to voice your frustration."""

rubric = """There are two criterias that you are creating these complaints on:

the tone harshness score on a scale of 1-3. 1 will be the least harsh, 2 will be a harsh and critical tone and 3 will be extremely harsh and confrontational.

the event harshness score on a scale of 1-3. 1 are mild issues that dont have a significant impact on the customer's experience. Examples are service disruptions, 
temporary delays in transactions, or other service errors. 2 are more significant issues that may have more of an impact on the customer's overall experience with the bank. 
Examples are unapproved charges, contesting transactions, money loss less that $10,000, social security-related activities, and debt collecting activities. 
3 are severe issues that may have significant impact on the customer's overall experience with the bank. Examples include fraudulent charges, fraud-related activities, 
lawsuit, hacking activities, money loss of more than $10,000."""
# task_textgen = f"Use this complaint below as an example of tone severity = {tone} and case/event severity = {case}.{complaint} I want you to write a complaint based on this one that de-escalates it to a case severity = ___ and a tone severity = ___. "


role_sent_analysis = """You are an analyst at a commercial bank tasked with analyzing customer complaints. 
The bank is required to respond to customer complaints in less than 10 days. As a result, you are tasked with scoring complaints for their 
case severity: severity of the their financial impact and tone severity: how harsh the complaint is.
Based on this criteria, provide a tone and case rating for the following complaint. Your output should be 3 characters long: two numbers seperated by a comma. An example output would be: 3,2


"""



# Escalation code below:

In [33]:
new_escalated_df = subset.copy()

# Initialize columns for escalated complaints
new_escalated_df['Escalated Complaint (Case Increased)'] = ''
new_escalated_df['Tone Severity - Constant'] = ''
new_escalated_df['Case Severity - Increased'] = ''
new_escalated_df['Sentiment Analysis - Complaint 1'] = ''
new_escalated_df['Escalated Complaint (Tone Increased)'] = ''
new_escalated_df['Tone Severity - Increased'] = ''
new_escalated_df['Case Severity - Constant'] = ''
new_escalated_df['Sentiment Analysis - Complaint 2'] = ''


# Loop through the rows of subset
for index, row in subset.iterrows():
    if row['Tone Severity'] < 3 and row['Case Severity'] < 3:
        
        # # if index < 5:
        # #     # For the first five rows, include the prompt and rubric in the input
        # input_case = role_textgen + rubric + task_case
        # input_tone = role_textgen + rubric + task_tone
        # # else:
        # #     # For the rest, only use the task
        # #     input_case = task_case
        # #     input_tone = task_tone

        #     # Call the chat_with_llm function with the constructed input
        # escalated_complaint_case = chat_with_llm(input_case)
        # escalated_complaint_tone = chat_with_llm(input_tone)
        
        # Increase Case Severity
        task_case = f"Use this complaint below as an example of tone severity = {row['Tone Severity']} and case/event severity = {row['Case Severity']}.{row['Random Complaint']} I want you to write a complaint based on this one that escalates it to a case severity = {row['Case Severity'] + 1} and a tone severity = {row['Tone Severity']}. "
        escalated_complaint_case = chat_with_llm(role_textgen + rubric + task_case)
        new_escalated_df.at[index, 'Escalated Complaint (Case Increased)'] = escalated_complaint_case
        new_escalated_df.at[index, 'Tone Severity - Constant'] = row['Tone Severity']
        new_escalated_df.at[index, 'Case Severity - Increased'] = row['Case Severity'] + 1


        # Increase Tone Severity
        task_tone = f"Use this complaint below as an example of tone severity = {row['Tone Severity']} and case/event severity = {row['Case Severity']}.{row['Random Complaint']} I want you to write a complaint based on this one that escalates it to a case severity = {row['Case Severity']} and a tone severity = {row['Tone Severity'] + 1}. "
        escalated_complaint_tone = chat_with_llm(role_textgen + rubric + task_case)
        new_escalated_df.at[index, 'Escalated Complaint (Tone Increased)'] = escalated_complaint_tone
        new_escalated_df.at[index, 'Tone Severity - Increased'] = row['Tone Severity'] + 1
        new_escalated_df.at[index, 'Case Severity - Constant'] = row['Case Severity']
       
    else:
        # Skip rows where either severity is already at maximum
        continue

# Display the first few rows of the new DataFrame
# new_escalated_df.head()


# De-escalation code below: 

In [27]:
new_deescalated_df = subset.copy()

# Initialize columns for de-escalated complaints
new_deescalated_df['De-escalated Complaint (Case Decreased)'] = ''
new_deescalated_df['Tone Severity - Constant'] = ''
new_deescalated_df['Case Severity - Decreased'] = ''
new_deescalated_df['Sentiment Analysis - Complaint 1'] = ''
new_deescalated_df['De-escalated Complaint (Tone Decreased)'] = ''
new_deescalated_df['Tone Severity - Decreased'] = ''
new_deescalated_df['Case Severity - Constant'] = ''
new_deescalated_df['Sentiment Analysis - Complaint 2'] = ''

# Loop through the rows of subset
for index, row in subset.iterrows():
    if row['Tone Severity'] > 1 and row['Case Severity'] > 1:

        # if index < 5:
        #     # For the first five rows, include the prompt and rubric in the input
        #     input_case = role_textgen + rubric + task_case
        #     input_tone = role_textgen + rubric + task_tone
        # else:
        #     # For the rest, only use the task
        #     input_case = task_case
        #     input_tone = task_tone

        #    # Call the chat_with_llm function with the constructed input
        # deescalated_complaint_case = chat_with_llm(input_case)
        # deescalated_complaint_tone = chat_with_llm(input_tone)


        # Decrease Case Severity
        task_case = f"Use this complaint below as an example of tone severity = {row['Tone Severity']} and case/event severity = {row['Case Severity']}.{row['Random Complaint']} I want you to write a complaint based on this one that de-escalates it to a case severity = {row['Case Severity'] - 1} and a tone severity = {row['Tone Severity']}. "
        deescalated_complaint_case = chat_with_llm(role_textgen + rubric + task_case)
        new_deescalated_df.at[index, 'De-escalated Complaint (Case Decreased)'] = deescalated_complaint_case
        new_deescalated_df.at[index, 'Tone Severity - Constant'] = row['Tone Severity']
        new_deescalated_df.at[index, 'Case Severity - Decreased'] = row['Case Severity'] - 1

        # Decrease Tone Severity
        task_tone = f"Use this complaint below as an example of tone severity = {row['Tone Severity']} and case/event severity = {row['Case Severity']}.{row['Random Complaint']} I want you to write a complaint based on this one that de-escalates it to a case severity = {row['Case Severity']} and a tone severity = {row['Tone Severity'] - 1}. "
        deescalated_complaint_tone = chat_with_llm(role_textgen + rubric + task_case)
        new_deescalated_df.at[index, 'De-escalated Complaint (Tone Decreased)'] = deescalated_complaint_tone
        new_deescalated_df.at[index, 'Tone Severity - Decreased'] = row['Tone Severity'] - 1
        new_deescalated_df.at[index, 'Case Severity - Constant'] = row['Case Severity']

    else:
        # Skip rows where de-escalation is not possible
        continue

# Display the first few rows of the new DataFrame
# new_deescalated_df.head()


In [ ]:
display(new_deescalated_df)

,Random Complaint,Tone Severity,Case Severity,De-escalated Complaint (Case Decreased),Tone Severity - Constant,Case Severity - Decreased,Sentiment Analysis - Complaint 1,De-escalated Complaint (Tone Decreased),Tone Severity - Decreased,Case Severity - Constant
0,"I have a Bank of America Mastercard. Today, XX...",1.0,1.0,,,,,,,
1,I used the Home Depot Credit card to make a pu...,1.0,2.0,,,,,,,


In [30]:
# ****** TODO: CHANGE THE MODEL NAME ******** 
escalated_csv_path = 'LLAMA_escalated50.csv'
new_escalated_df.to_csv(escalated_csv_path, index=False)

# ****** TODO: CHANGE THE MODEL NAME ******** 
deescalated_csv_path = 'LLAMA_deescalated50.csv'
new_deescalated_df.to_csv(deescalated_csv_path, index=False)

# Return the paths to the CSV files for download
escalated_csv_path, deescalated_csv_path

# Export to Excel
# excel_filename = '/complaints_severity_analysis.xlsx'
# with pd.ExcelWriter(excel_filename, engine='xlsxwriter') as writer:  
#     new_escalated_df.to_excel(writer, sheet_name='Escalated Complaints', index=False)
#     new_deescalated_df.to_excel(writer, sheet_name='De-escalated Complaints', index=False)
#     writer.save()

# Provide the path to the saved Excel file
# excel_filename

('LLAMA_escalated50.csv', 'LLAMA_deescalated50.csv')

# Sentiment Analysis Section: YOU FIRST NEED TO CHANGE THE MODEL YOU ARE USING IN LM STUDIO

If you are using Llama for text gen, then perform sentiment analysis using Mistral and vice versa. 

In [ ]:
# Loop through the rows of subset
for index, row in subset.iterrows():
    if row['Tone Severity'] < 3 and row['Case Severity'] < 3:
        
        if index < 5:
            # For the first five rows, include the prompt and rubric in the input
            input_case = role_textgen + rubric + task_case
            input_tone = role_textgen + rubric + task_tone
        else:
            # For the rest, only use the task
            input_case = task_case
            input_tone = task_tone

            # Call the chat_with_llm function with the constructed input
        escalated_complaint_case = chat_with_llm(input_case)
        escalated_complaint_tone = chat_with_llm(input_tone)
        
        # Increase Case Severity
        task_case = f"Use this complaint below as an example of tone severity = {row['Tone Severity']} and case/event severity = {row['Case Severity']}.{row['Random Complaint']} I want you to write a complaint based on this one that escalates it to a case severity = {row['Case Severity'] + 1} and a tone severity = {row['Tone Severity']}. "
        # escalated_complaint_case = chat_with_llm(role_textgen + rubric + task_case)
        new_escalated_df.at[index, 'Escalated Complaint (Case Increased)'] = escalated_complaint_case
   
       
    else:
        # Skip rows where either severity is already at maximum
        continue

# Display the first few rows of the new DataFrame
# new_escalated_df.head()


In [ ]:
role_sent_analysis = f"""You are an analyst at a commercial bank tasked with analyzing customer complaints. 
The bank is required to respond to customer complaints in less than 10 days. As a result, you are tasked with scoring complaints for their 
case severity: severity of the their financial impact and tone severity: how harsh the complaint is.{rubric}
Based on this criteria, provide a tone and case rating for the following complaint. {subset['Random Complaint'][1]} Your output start with your scores and should be two numbers seperated by a comma. An example output would be: 3,2. Do not provide an explanation"""

chat_with_llm(role_sent_analysis)

In [ ]:
def process_complaint(row, role, rubric, escalation=True):
    tone = row['Tone Severity']
    case = row['Case Severity']
    complaint = row['Random Complaint']
    
    if escalation:
        # Check if we can escalate
        if tone < 3 and case < 3:
            new_case = case + 1
            new_tone = tone  # tone stays the same
    else:
        # Check if we can de-escalate
        if tone > 1 and case > 1:
            new_case = case  # case stays the same
            new_tone = tone - 1

    # Prepare the task string
    task = f"Use this complaint below as an example of tone severity = {tone} and case/event severity = {case}.{complaint} I want you to write a complaint based on this one that escalates it to a case severity = {new_case} and a tone severity = {new_tone}. "

    # Call the LLM function
    return chat_with_llm(role + rubric + task)


In [ ]:

# Assuming 'subset' is your DataFrame with complaints
# Create a copy to hold new escalated complaints
new_escalated_df = subset_test.sample(3).copy()

# Process the complaints for escalation
new_escalated_df['Escalated Complaint'] = new_escalated_df.apply(lambda row: process_complaint(row, role, rubric, escalation=True) if row['Tone Severity'] < 3 and row['Case Severity'] < 3 else "", axis=1)

# Create a copy to hold new de-escalated complaints
new_deescalated_df = subset_test.sample(3).copy()

# Process the complaints for de-escalation
new_deescalated_df['De-escalated Complaint'] = new_deescalated_df.apply(lambda row: process_complaint(row, role, rubric, escalation=False) if row['Tone Severity'] > 1 and row['Case Severity'] > 1 else "", axis=1)
